Data selection 기준
1. 유저가 5번 이상 들은 노래를 선호하는 노래로 가정하고 유저가 선호하는 노래 추출
2. 1000명 이상의 유저가 선호하는 노래 추출
3. tag, genre, uri 정보가 없는 데이터 제외
4. 선호하는 노래가 50개 이상인 유저 추출

In [1]:
import pandas as pd
import json

### 1. 유저가 5번 이상 들은 노래를 선호하는 노래로 가정하고 유저가 선호하는 노래 추출

In [2]:
# listening-counts.tsv 로드 (+ chunk 설정, type 설정)
chunk_size = 1000000
chunks = []
df_dtype = {'user_id': 'int32', 'track_id': 'int32', 'count': 'int16'}
for i, chunk in enumerate(pd.read_csv('./data/listening-counts.tsv', sep='\t', dtype=df_dtype, chunksize=chunk_size)):
    chunks.append(chunk)
listening = pd.concat(chunks, ignore_index=True)
listening

,user_id,track_id,count
0,92915,26719936,1
1,92915,4271407,1
2,92915,4606511,1
3,92915,8966085,1
4,92915,12290897,1
...,...,...,...
519293328,119080,44144633,1
519293329,119080,45141923,1
519293330,119080,46171716,1
519293331,119080,46245572,1


In [3]:
# 5번 이상 들은 track이 user가 선호하는 track이라고 가정
listening = listening[listening['count']>=5]
listening = listening.reset_index(drop=True)
listening

,user_id,track_id,count
0,92915,34145565,24
1,92915,41415266,6
2,92915,44859826,8
3,92915,6816877,18
4,92915,5020507,21
...,...,...,...
91767600,119080,44184032,5
91767601,119080,45065482,5
91767602,119080,11791597,9
91767603,119080,22273648,5


### 2. 1000명 이상의 유저가 선호하는 노래 추출

In [4]:
# 각 track을 선호하는 user 수 계산
listening_count = listening.groupby('track_id').agg({'user_id': 'count', 'count': 'sum'})
listening_count.columns = ['n_user', 'total_listening']
listening_count

,n_user,total_listening
track_id,,
4,1,6
52,1,6
56,1,10
64,1,32
69,1,26
...,...,...
50813261,1,5
50813274,1,5
50813284,12,125


In [5]:
# 1000명 이상의 user가 선호하는 track 추출
listening_count = listening_count[listening_count['n_user']>=1000]
listening_count = listening_count.sort_values('n_user', ascending=False)
listening_count = listening_count.reset_index()
listening_count

,track_id,n_user,total_listening
0,36346257,12473,252744
1,33619193,9713,193666
2,36039983,9638,172650
3,26445594,9246,184883
4,32083560,9078,184951
...,...,...,...
5833,32144610,1000,15783
5834,32015006,1000,13893
5835,18665821,1000,15332
5836,38006695,1000,15063


### 3. tag, genre, uri 정보가 없는 데이터 제외

In [6]:
# tracks.tsv
df_dtype = {'track_id': 'int32'}
tracks = pd.read_csv('./data/tracks.tsv', sep='\t', dtype=df_dtype, on_bad_lines='skip')

# artists.tsv
artists = pd.read_csv('./data/artists.tsv', sep='\t')

# tags.json
tags = []
with open('./data/tags.json', encoding='utf-8') as f:
    for line in f:
        tag = json.loads(line)
        tags.append(tag)
tags = pd.DataFrame(tags)

# tags-micro-genres.json
genres = []
with open('./data/tags-micro-genres.json', encoding='utf-8') as f:
    for line in f:
        tag = json.loads(line)
        genres.append(tag)
genres = pd.DataFrame(genres)
genres = genres.rename(columns={'tags':'genres'})

# spotify-uris.tsv
uris = pd.read_csv('./data/spotify-uris.tsv', sep='\t')

In [7]:
sideinfo = pd.merge(listening_count, tracks, how='left',on='track_id')
sideinfo = pd.merge(sideinfo, tags[['i','tags']], how='left', left_on='track_id', right_on='i').drop(columns='i')
sideinfo = pd.merge(sideinfo, genres[['i','genres']], how='left', left_on='track_id', right_on='i').drop(columns='i')
sideinfo = pd.merge(sideinfo, uris, how='left',on='track_id')
sideinfo

,track_id,n_user,total_listening,artist_name,track_name,tags,genres,uri
0,36346257,12473,252744,Gotye,Somebody That I Used to Know,"{'indie': 100, 'alternative': 79, 'electronic'...","{'pop': 19, 'indie pop': 10, 'rock': 5, 'singe...",4wCmqSrbyCgxEXROQE6vtV
1,33619193,9713,193666,Adele,Rolling in the Deep,"{'soul': 100, 'Adele': 95, 'female vocalists':...","{'soul': 100, 'pop': 69, 'singer-songwriter': ...",1c8gk2PeTE04A1pIDH9YMk
2,36039983,9638,172650,Nirvana,Smells Like Teen Spirit,"{'Grunge': 100, 'rock': 69, 'Nirvana': 43, '90...","{'grunge': 100, 'rock': 69, 'alternative rock'...",5ghIJDpPoe3CfHMGu71E6T
3,26445594,9246,184883,M83,Midnight City,"{'electronic': 100, 'indie': 60, 'electropop':...","{'electropop': 49, 'electro': 31, 'synthpop': ...",1eyzqe2QqGZUmfcPZtrIyt
4,32083560,9078,184951,Foster the People,Pumped Up Kicks,"{'indie': 100, 'indie pop': 69, 'catchy': 38, ...","{'indie pop': 69, 'pop': 17, 'indie rock': 17,...",7w87IxuO7BDcJ3YUqCyMTT
...,...,...,...,...,...,...,...,...
5833,32144610,1000,15783,Static-X,Push It,"{'industrial metal': 100, 'metal': 79, 'Nu Met...","{'industrial metal': 100, 'metal': 79, 'nu met...",6O7pihLJgeqxUnG7u9oYL2
5834,32015006,1000,13893,Massive Attack,Psyche,"{'trip-hop': 100, 'electronic': 62, 'trip hop'...","{'trip hop': 37, 'electronica': 20, 'alternati...",5HZrnSHCqaazNzr5w1enKG
5835,18665821,1000,15332,Kanye West,Homecoming,"{'Hip-Hop': 100, 'Kanye West': 77, 'rap': 75, ...","{'rap': 75, 'hip hop': 44, 'pop': 6, 'metal': ...",2ovQ5MCx91XVjgVWEPfvks
5836,38006695,1000,15063,Kyuss,Supa Scoopa and Mighty Scoop,"{'Stoner Rock': 100, 'desert rock': 43, 'stone...","{'stoner rock': 100, 'rock': 29, 'stoner metal...",74PX4Sa0tXJDStditawG0O


In [8]:
# 결측치 확인
print('tags :', sideinfo['tags'].isnull().sum())
print('genres :', sideinfo['genres'].isnull().sum())
print('uri :', sideinfo['uri'].isnull().sum())
sideinfo[sideinfo.isnull().any(axis=1)]

tags : 7
genres : 9
uri : 178


,track_id,n_user,total_listening,artist_name,track_name,tags,genres,uri
99,44249371,5128,98602,Rihanna,We Found Love (feat. Calvin Harris),"{'pop': 100, 'dance': 98, 'Rihanna': 67, 'fema...","{'pop': 100, 'electropop': 10, 'house': 10, 'd...",NaN
120,8457633,4840,80339,Red Hot Chili Peppers,Can't Stop,"{'rock': 100, 'alternative rock': 64, 'funk': ...","{'rock': 100, 'alternative rock': 64, 'funk': ...",NaN
128,12494556,4788,83844,Journey,Don't Stop Believin',"{'classic rock': 100, '80s': 95, 'rock': 65, '...","{'classic rock': 100, 'rock': 65, 'pop': 9, 'h...",NaN
130,12496203,4771,81450,Queen,Don't Stop Me Now,"{'classic rock': 100, 'rock': 85, 'Queen': 70,...","{'classic rock': 100, 'rock': 85, 'glam rock':...",NaN
133,45985119,4737,82802,Florence + the Machine,You've Got the Love,"{'indie': 100, 'female vocalists': 84, 'altern...","{'indie pop': 24, 'pop': 16, 'soul': 11, 'rock...",NaN
...,...,...,...,...,...,...,...,...
5761,12492451,1008,14690,Michael Jackson,Don't Stop 'Til You Get Enough,"{'pop': 100, 'Disco': 63, 'dance': 47, '80s': ...","{'pop': 100, 'disco': 63, 'funk': 27, 'soul': ...",NaN
5795,19822110,1004,14148,Nelly Furtado,I'm Like a Bird,"{'pop': 100, 'female vocalists': 50, 'Nelly Fu...","{'pop': 100, 'folk': 9, 'singer-songwriter': 8...",NaN
5826,12440113,1001,21938,Bring Me the Horizon,Don't Go,"{'metalcore': 100, 'deathcore': 75, 'screamo':...","{'metalcore': 100, 'deathcore': 75, 'screamo':...",NaN
5828,13296088,1001,17259,30 Seconds to Mars,Edge of the Earth,"{'alternative rock': 100, 'rock': 75, '30 seco...","{'alternative rock': 100, 'rock': 75, 'progres...",NaN


In [9]:
# 결측치가 존재하는 행 삭제
sideinfo = sideinfo.dropna().reset_index(drop=True)
sideinfo

,track_id,n_user,total_listening,artist_name,track_name,tags,genres,uri
0,36346257,12473,252744,Gotye,Somebody That I Used to Know,"{'indie': 100, 'alternative': 79, 'electronic'...","{'pop': 19, 'indie pop': 10, 'rock': 5, 'singe...",4wCmqSrbyCgxEXROQE6vtV
1,33619193,9713,193666,Adele,Rolling in the Deep,"{'soul': 100, 'Adele': 95, 'female vocalists':...","{'soul': 100, 'pop': 69, 'singer-songwriter': ...",1c8gk2PeTE04A1pIDH9YMk
2,36039983,9638,172650,Nirvana,Smells Like Teen Spirit,"{'Grunge': 100, 'rock': 69, 'Nirvana': 43, '90...","{'grunge': 100, 'rock': 69, 'alternative rock'...",5ghIJDpPoe3CfHMGu71E6T
3,26445594,9246,184883,M83,Midnight City,"{'electronic': 100, 'indie': 60, 'electropop':...","{'electropop': 49, 'electro': 31, 'synthpop': ...",1eyzqe2QqGZUmfcPZtrIyt
4,32083560,9078,184951,Foster the People,Pumped Up Kicks,"{'indie': 100, 'indie pop': 69, 'catchy': 38, ...","{'indie pop': 69, 'pop': 17, 'indie rock': 17,...",7w87IxuO7BDcJ3YUqCyMTT
...,...,...,...,...,...,...,...,...
5633,32144610,1000,15783,Static-X,Push It,"{'industrial metal': 100, 'metal': 79, 'Nu Met...","{'industrial metal': 100, 'metal': 79, 'nu met...",6O7pihLJgeqxUnG7u9oYL2
5634,32015006,1000,13893,Massive Attack,Psyche,"{'trip-hop': 100, 'electronic': 62, 'trip hop'...","{'trip hop': 37, 'electronica': 20, 'alternati...",5HZrnSHCqaazNzr5w1enKG
5635,18665821,1000,15332,Kanye West,Homecoming,"{'Hip-Hop': 100, 'Kanye West': 77, 'rap': 75, ...","{'rap': 75, 'hip hop': 44, 'pop': 6, 'metal': ...",2ovQ5MCx91XVjgVWEPfvks
5636,38006695,1000,15063,Kyuss,Supa Scoopa and Mighty Scoop,"{'Stoner Rock': 100, 'desert rock': 43, 'stone...","{'stoner rock': 100, 'rock': 29, 'stoner metal...",74PX4Sa0tXJDStditawG0O


In [10]:
# 메모리 확보를 위해 정의된 변수 확인
%whos

Variable          Type             Data/Info
--------------------------------------------
artists           DataFrame                 artist_id       <...>5159213 rows x 2 columns]
chunk             DataFrame                   user_id  track<...>[293333 rows x 3 columns]
chunk_size        int              1000000
chunks            list             n=520
df_dtype          dict             n=1
f                 TextIOWrapper    <_io.TextIOWrapper name='<...>ode='r' encoding='utf-8'>
genres            DataFrame                                 <...>1638468 rows x 3 columns]
i                 int              519
json              module           <module 'json' from 'c:\\<...>\lib\\json\\__init__.py'>
line              str              {"_id":{"artist":"Alan Pa<...>bient":2},"i":14456757}\n
listening         DataFrame                  user_id  track_<...>1767605 rows x 3 columns]
listening_count   DataFrame              track_id  n_user  t<...>\n[5838 rows x 3 columns]
pd                mo

In [11]:
# 필요없는 변수 제거
del listening_count, tracks, artists, tags, genres, uris

### 4. 선호하는 노래가 50개 이상인 유저 추출

In [12]:
# 추출한 track 리스트
track_list = sorted(sideinfo['track_id'])
print(len(track_list))
print(track_list[:5], track_list[-5:])

5638
[24587, 126616, 148113, 168359, 181098] [47820385, 47895780, 47924598, 47940129, 49707344]


In [13]:
# listening 데이터에서 track 리스트에 대한 데이터만 추출 후 정렬
interaction = listening[['user_id', 'track_id']]
interaction = interaction[interaction['track_id'].isin(track_list)]
interaction = interaction.sort_values(['user_id', 'track_id'])
interaction = interaction.reset_index(drop=True)
interaction

,user_id,track_id
0,0,4835013
1,0,5114312
2,0,6100597
3,0,6735576
4,0,7308840
...,...,...
10228415,120321,45915927
10228416,120321,46831592
10228417,120321,47111095
10228418,120321,47702931


In [14]:
# 선호하는 노래가 50개 이상인 user 리스트
user_track_count = interaction.groupby('user_id').count()
user_list = list(user_track_count[user_track_count['track_id'] >= 50].index)
print(len(user_list))
print(user_list[:5], user_list[-5:])

44668
[0, 2, 3, 5, 6] [120292, 120313, 120315, 120319, 120321]


In [15]:
# interaction 데이터에서 선호하는 노래가 50개 이상인 데이터 추출
interaction = interaction[interaction['user_id'].isin(user_list)]
interaction = interaction.reset_index(drop=True)
interaction

,user_id,track_id
0,0,4835013
1,0,5114312
2,0,6100597
3,0,6735576
4,0,7308840
...,...,...
9483351,120321,45915927
9483352,120321,46831592
9483353,120321,47111095
9483354,120321,47702931


In [16]:
# interaction 데이터에 없는 track sideinfo에서 제거
track_list = interaction['track_id'].unique()
sideinfo = sideinfo[sideinfo['track_id'].isin(track_list)]
sideinfo = sideinfo.reset_index(drop=True)
sideinfo

,track_id,n_user,total_listening,artist_name,track_name,tags,genres,uri
0,36346257,12473,252744,Gotye,Somebody That I Used to Know,"{'indie': 100, 'alternative': 79, 'electronic'...","{'pop': 19, 'indie pop': 10, 'rock': 5, 'singe...",4wCmqSrbyCgxEXROQE6vtV
1,33619193,9713,193666,Adele,Rolling in the Deep,"{'soul': 100, 'Adele': 95, 'female vocalists':...","{'soul': 100, 'pop': 69, 'singer-songwriter': ...",1c8gk2PeTE04A1pIDH9YMk
2,36039983,9638,172650,Nirvana,Smells Like Teen Spirit,"{'Grunge': 100, 'rock': 69, 'Nirvana': 43, '90...","{'grunge': 100, 'rock': 69, 'alternative rock'...",5ghIJDpPoe3CfHMGu71E6T
3,26445594,9246,184883,M83,Midnight City,"{'electronic': 100, 'indie': 60, 'electropop':...","{'electropop': 49, 'electro': 31, 'synthpop': ...",1eyzqe2QqGZUmfcPZtrIyt
4,32083560,9078,184951,Foster the People,Pumped Up Kicks,"{'indie': 100, 'indie pop': 69, 'catchy': 38, ...","{'indie pop': 69, 'pop': 17, 'indie rock': 17,...",7w87IxuO7BDcJ3YUqCyMTT
...,...,...,...,...,...,...,...,...
5633,32144610,1000,15783,Static-X,Push It,"{'industrial metal': 100, 'metal': 79, 'Nu Met...","{'industrial metal': 100, 'metal': 79, 'nu met...",6O7pihLJgeqxUnG7u9oYL2
5634,32015006,1000,13893,Massive Attack,Psyche,"{'trip-hop': 100, 'electronic': 62, 'trip hop'...","{'trip hop': 37, 'electronica': 20, 'alternati...",5HZrnSHCqaazNzr5w1enKG
5635,18665821,1000,15332,Kanye West,Homecoming,"{'Hip-Hop': 100, 'Kanye West': 77, 'rap': 75, ...","{'rap': 75, 'hip hop': 44, 'pop': 6, 'metal': ...",2ovQ5MCx91XVjgVWEPfvks
5636,38006695,1000,15063,Kyuss,Supa Scoopa and Mighty Scoop,"{'Stoner Rock': 100, 'desert rock': 43, 'stone...","{'stoner rock': 100, 'rock': 29, 'stoner metal...",74PX4Sa0tXJDStditawG0O


In [17]:
print('n_user :', len(interaction['user_id'].unique()))
print('n_item :', len(interaction['track_id'].unique()))
print('n_interaction :', len(interaction))

n_user : 44668
n_item : 5638
n_interaction : 9483356


### Save csv

In [18]:
interaction.to_csv('./data/interaction.csv')

In [19]:
sideinfo.to_csv('./data/sideinfo.csv')

### Save interaction.json (용량 감소)

In [ ]:
interaction_json = []
for _, df_user in interaction.groupby('user_id'):
    interaction_json.append({'user_id': int(df_user['user_id'].iloc[0]), 'track_id': list(df_user['track_id'])})
interaction_json[0]

In [ ]:
with open('./data/interaction.json', 'w') as f:
    json.dump(interaction_json, f)